In [3]:
import requests
from bs4 import BeautifulSoup
import select
import sqlite3
import time
import selenium
from selenium import webdriver

In [4]:
driver = webdriver.Chrome()

In [5]:
con = sqlite3.connect("news.db")

In [6]:
cur = con.cursor()

In [7]:
cur.executescript("""
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        url TEXT NOT NULL
    );
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table1_id INTEGER NOT NULL,
        content TEXT NOT NULL
    );
""")

In [8]:
driver.get("https://news.naver.com")

In [9]:
# main_content > div > .com_list li > a

In [10]:
for _ in driver.find_elements_by_css_selector("#main_content > div > .com_list li > a"):
    if len(_.get_attribute("href")) > 1:
        cur.execute("INSERT INTO table1(title, url) VALUES(?, ?)",[_.text, _.get_attribute("href")])
        con.commit()

--> Comprehension

In [19]:
[cur.execute("INSERT INTO table1(title, url) VALUES(?, ?)",[_.text, _.get_attribute("href")]) 
 for _ in driver.find_elements_by_css_selector("#main_content > div > .com_list li > a")
    if len(_.get_attribute("href")) > 1]

In [20]:
con.commit()

In [70]:
cur.execute("SELECT * FROM table1")

위 쿼리 insert는 왜 알아서 30개까지만 되지?-> 여러번하면 31번부터 60번, 61번부터 90번

_에 담긴 list를 초기화하려면 어떡하지?

In [57]:
for _ in cur.fetchall():
   #print(driver.get(_))   
    driver.get(_[2])
    
    cur.execute("INSERT INTO table2(table1_id, content) VALUES(?, ?)",
               [_[0], driver.find_element_by_css_selector("#articleBodyContents").text])
    con.commit()
    

-> Comprehension

In [ ]:
[cur.execut"INSERT INTO table2(table1_id, content) VALUES(?,?)",
            [_[0],driver.find_element_by_css_selector("#articleBodyContents").text]) 
 for _ in [row for row in cur.fetchall()]]
# 람다쓰로 한번

In [73]:
insertlist = [lambda x: ("INSERT INTO table2(table1_id) VALUES("+x[0]+")",x[2])
 for x in [row for row in cur.fetchall()]]

In [74]:
insertlist

[<function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function

In [72]:
[driver.get(_[1]) for _ in insertlist]

TypeError: 'function' object is not subscriptable

In [52]:
con.commit()

In [42]:
[row for row in cur.fetchall()]

[(31,
  '돌아온 홍영표 "다음주부터 정개특위 가동…원안 고집 안해"',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=079&aid=0003250307'),
 (32,
  '단식 조롱, 바른미래당 내홍 격화 “손학규 주변인 도 넘은 비난”',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=005&aid=0001219694'),
 (33,
  '文지지율 50%대 회복…대일 강경모드 효과?',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=079&aid=0003250304'),
 (34,
  '중재위 오늘 시한…日 언론 “韓정부 불응에 타개책 안 보여”',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=374&aid=0000189254'),
 (35,
  "이인영 '결자해지론'에 정개특위 위원장 수락한 홍영표",
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=025&aid=0002923185'),
 (36,
  '금융당국 “외화보험 주의해야…환율 따라 보험금도 변동”',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=374&aid=0000189264'),
 (37,
  '한국 온 유학생 ‘사상 최대’…한국인 유학 붐은 감소',
  'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=469&aid=0000406521'),
 (38,
  '‘화재가능성 발견’ 현대차 제네시스 등 12개 차종 리콜',
  'https://news.

---

linked in - out scraping 

In [ ]:
for _ in [_ for _ in driver.find_elements_by_css_selector("#main_content > div > .com_list li > a")
         if len(_.get_attribute("href")) > 1]:
    news = (_.text, _.get_attribute("href"))
    
    cur.execute("INSERT INTO table1(title, url) VALUES(?,?)",[news[0], news[1]])
    con.commit()
    
    cur.execute("SELECT id FROM table1 WHERE url=?", [news[1]])
    newsID = cur.fetchone()[0]
    
    driver.execute_script("window.open('about:blank');")
    driver.execute_script("console.log('%$');" % news[0].replace('\'','\"'))
    
    driver.switch_to_window(driver.window_handles[-1])
    driver.get(news[1])
    
    cur.execute("INSERT INTO table2(table1_id, content) VALUES(?, ?)",
                [newsID, driver.find_element_by_css_selector("#articleBodyContents").text])
    con.commit()
    
    driver.close()
    driver.switch_to_window(driver.window_handles[0])
    
driver.close()